In [1]:
import pandas as pd
import MetaTrader5 as mt5
from sup_res import zig_zag,zig_zag_data,full_table,zig_zag_2,zig_zag_tsl,zig_zag_100
import warnings
warnings.filterwarnings("ignore")

if not mt5.initialize(path = r'C:\Program Files\Zero Markets MetaTrader 5 Terminal\terminal64.exe'):
    print("initialize() failed")
    mt5.shutdown()

In [2]:
ativo = 'EURUSD'
df = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M5, 0, 40000))
df['Data'] = pd.to_datetime(df['time'], unit='s').apply(lambda x: str(x))
df = df[['Data','open','high','low','close','real_volume']]
df.columns = ['Data','Open','High','Low','Close','Vol']
df = df.rename(columns = {'real_volume':'vol'})

In [3]:
timeframe = 5
new_total = None
for merge_percent in [0.1,0.2,0.3,0.4,0.5,0.6]:
    for zig_zag_percent in [0.1,0.2,0.3,0.4,0.5,0.6]:
        for rate_tp in [1]:
            for pontos in [0.0006]:

                stats = zig_zag_data(df,zig_zag_percent=zig_zag_percent, merge_distance=None,
                                    merge_percent=merge_percent, min_bars_between_peaks=10, peaks='All',
                                    pontos = pontos,rate_tp = rate_tp, rsi = [14,70,30])
                
                new = full_table(stats, pontos, rate_tp,rate_stop = 1,spread=0.00005,
                timeframe = timeframe,zig_zag_percent=zig_zag_percent,merge_percent=merge_percent)

                new_total = pd.concat([new_total,new])

                print(new_total.to_string())


   Timeframe  pontos  zig_zag_percent  merge_percent  rate_tp  N_trades  Taxa_acerto  Pontos_total   Retorno
0          5  0.0006              0.1            0.1        1       234    52.564103       -0.0045  0.785185
   Timeframe  pontos  zig_zag_percent  merge_percent  rate_tp  N_trades  Taxa_acerto  Pontos_total   Retorno
0          5  0.0006              0.1            0.1        1       234    52.564103       -0.0045  0.785185
0          5  0.0006              0.2            0.1        1       214    60.280374        0.0157  2.742518
   Timeframe  pontos  zig_zag_percent  merge_percent  rate_tp  N_trades  Taxa_acerto  Pontos_total   Retorno
0          5  0.0006              0.1            0.1        1       234    52.564103       -0.0045  0.785185
0          5  0.0006              0.2            0.1        1       214    60.280374        0.0157  2.742518
0          5  0.0006              0.3            0.1        1       178    56.741573        0.0055  1.528045
   Timeframe  ponto

In [3]:
new_total.sort_values(by='Taxa_acerto', ascending=False).head(50)

AttributeError: 'NoneType' object has no attribute 'sort_values'

In [4]:
new_total.to_csv('eurusd5_candlepatterns_4.csv',index = False)

AttributeError: 'NoneType' object has no attribute 'to_csv'

In [5]:
df['Data_d'] = df['Data'].apply(lambda x: x[:10])

lista_data = df['Data_d'].unique()

df_new = df[df['Data_d'] == lista_data[3]]
df_2 = df[df['Data_d'] == lista_data[4]]

NameError: name 'df' is not defined

In [6]:
plot_zig_zag(df_new,df_2,zig_zag_percent=0.2, merge_distance=None,
                         merge_percent=0.3, min_bars_between_peaks=10, peaks='All')

NameError: name 'plot_zig_zag' is not defined

In [ ]:
df['Data_d'] = df['Data'].apply(lambda x: x[:10])

lista_data = df['Data_d'].unique()


df_total = None

for num,data in enumerate(lista_data):

    try:
        df_new = df[df['Data_d'] == lista_data[num]]
        df_2 = df[df['Data_d'] == lista_data[num + 1]]

        levels = zig_zag(df_new,zig_zag_percent=0.2, merge_distance=None,
                                merge_percent=0.3, min_bars_between_peaks=10, peaks='All')

        lista_price = [x['price'] for x in levels]

        df_2['prox_linha'] = df_2['Close'].apply(lambda x: min(lista_price, key=lambda y:abs(y-x)))

        df_total = pd.concat([df_total,df_2])

    except:

        pass






In [ ]:
df_total['Data'] = pd.to_datetime(df_total['Data'])
df_total = df_total.set_index('Data')


In [ ]:
#sup = 2, res = -2
#Verificar Win rate pra Low, High e Close. Close é cruzamento, Low,High é encostar
#estrategia 1 -> Cruzar no sup_res, entrar no candle que cruzou
#estrategia 2 -> Cruzar no sup_res, entrar proximo candle revertendo 
#estrategia 3 -> Toque no sup_res e proximo candle revertendo (inclusive pode ser o próprio candle) 

'''df_total['sup_res'] = df_total[['Close','prox_linha']].apply(lambda x: 2 if (x['Close'] - x['prox_linha'] < 0.0004) and (x['Close'] - x['prox_linha'] > 0) 
                                                        else -2 if (x['Close'] - x['prox_linha'] < -0.0004) and (x['Close'] - x['prox_linha'] < 0) 
                                                        else 1 if (abs(x['Close'] - x['prox_linha']) > 0.0004)
                                                        else 0, axis = 1)'''
df_total ['diff'] = df_total['Close'] - df_total['prox_linha']
df_total['prox_linha_ant'] = df_total['prox_linha'].shift(1)

df_total['sup'] = df_total[['Close','prox_linha','prox_linha_ant']].apply(lambda x: 1 if (x['Close'] - x['prox_linha'] > 0)
                                                            else -1 if (x['Close'] - x['prox_linha'] < 0) and
                                                            (x['prox_linha'] == x['prox_linha_ant'])
                                                            else 0,axis = 1)

df_total['res'] = df_total[['Close','prox_linha','prox_linha_ant']].apply(lambda x: 2 if (x['Close'] - x['prox_linha'] < 0)
                                                            else -1 if (x['Close'] - x['prox_linha'] > 0) and
                                                            (x['prox_linha'] == x['prox_linha_ant'])
                                                            else 0,axis = 1)



# 1--> Cruzou pra baixo, 2 --> Cruzou pra baixo 
df_total['cruzamento_sup'] = df_total['sup'].diff().apply(lambda x: 1 if x == -2 else 0)
df_total['cruzamento_res'] = df_total['res'].diff().apply(lambda x: 2 if x == -3 else 0)
#df_total['cruzamento'] = df_total['sup_res'].diff().apply(lambda x: 1 if x == -2 else 2 if x == -3 else 0)

df_total['signal'] = 0
for index, row in df_total.iterrows():
    if row['cruzamento_sup'] == 1:
        df_total.loc[index,'signal'] = row['cruzamento_sup']
    elif row['cruzamento_res'] == 2:
        df_total.loc[index,'signal'] = row['cruzamento_res']

#estrategia 1
#df_total['acao'] = df_total['cruzamento'].apply(lambda x: 'call' if x == 1 else 'sell' if x == 2 else 0)

#df_total['signal'] = df_total['cruzamento']

lista = df_total['signal'].values
tempo_lista =15
for num, i in enumerate(lista):
    #print(num,i)
    if i != 0:
        for i in range(tempo_lista):
            try:
                n_num = num + i + 1
                lista[n_num] = 0
            except:
                pass

pontos = 0.0008
rate_tp = 1

def SIGNAL():
        return df_total.signal

class MyCandlesStrat(Strategy):
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next() 
        sl = pontos
        tp = pontos * rate_tp
        if self.signal1==1:
            sl1 = self.data.Close[-1] - sl
            tp1 = self.data.Close[-1] + tp
            self.buy(sl=sl1, tp=tp1)
        elif self.signal1==2:
            sl1 = self.data.Close[-1] + sl
            tp1 = self.data.Close[-1] - tp
            self.sell(sl=sl1, tp=tp1)

bt = Backtest(df_total, MyCandlesStrat, cash=100000000, commission=0,hedging=True)
stats = bt.run()
bt.plot()

In [ ]:
df_trades = stats['_trades']
#df_trades = df_trades.drop_duplicates('EntryTime')
df_trades['Hora_h'] = df_trades['EntryTime'].apply(lambda x: x.hour)
df_trades['resultado_binario'] = df_trades['ReturnPct'].apply(lambda x: 1 if x > 0 else 0)

#print(f'Time_period: {time_period} -- var_bb: {var_bb} -- pontos: {pontos}--Rate_tp: {rate_tp}')
df_new_hora = df_trades[['Hora_h','resultado_binario']]
df_new_hora = df_trades.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

#df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
print(df_new_hora)

In [ ]:
df_total[['Close','prox_linha','diff','sup','cruzamento_sup', 'signal']].iloc[0:50]

In [ ]:
df_2[df_2['res'] == -3]